In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.models as models
import torch.utils.data as data
import torch.nn.functional as F

import numpy as np

## tensor variable parameter

In [2]:
# tensor
# variable, wraps a tensor, records the operations applied to it, and holds the gradient
# parameter, a kind of Variable

t = torch.randn(10,10)
v = Variable(t)
p = nn.Parameter(t)

o = v + p

print(t.size(), v.size(), p.size())
print(v.requires_grad, p.requires_grad)
print(v.grad, p.grad)
print(v.grad_fn, p.grad_fn)
print(v.is_leaf, p.is_leaf)

print(o.grad_fn)

torch.Size([10, 10]) torch.Size([10, 10]) torch.Size([10, 10])
False True
None None
None None
True True


## dataset

In [3]:
class Dataset(data.Dataset):

    def __init__(self, ):
        
        self.imgs = np.random.randn(100, 3, 224, 224).astype(np.float32)
        self.labs = np.random.randint(0, 10, (100,)).astype(np.float32)
    
    def __len__(self):
        
        return self.imgs.shape[0]
    
    def __getitem__(self, i):
        
        pass
    
        return self.imgs[i], self.labs[i]
  

In [4]:
dataset = Dataset()
dataloader = data.DataLoader(dataset, batch_size=20, num_workers=3, shuffle=True)

In [5]:
for i, (img, lab) in enumerate(dataloader):
    print(i, img.size(), lab.size())
    
print(type(img))

0 torch.Size([20, 3, 224, 224]) torch.Size([20])
1 torch.Size([20, 3, 224, 224]) torch.Size([20])
2 torch.Size([20, 3, 224, 224]) torch.Size([20])
3 torch.Size([20, 3, 224, 224]) torch.Size([20])
4 torch.Size([20, 3, 224, 224]) torch.Size([20])
<class 'torch.FloatTensor'>


## module

In [6]:
class NewModule(nn.Module):
    
    def __init__(self, input_cs):
        super(NewModule, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=input_cs, out_channels=32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 3, 2)
        
        self.model = nn.Sequential( self.conv1, nn.ReLU(), self.conv2, nn.ReLU() )
        
    def forward(self, x):
        
        out1 = F.relu( self.conv1(x) )
        
        out2 = self.model(x)
        
        return out1, out2

In [7]:
mm = NewModule(3)
x = Variable( torch.randn(10, 3, 224, 224) )
o1, o2 = mm(x)

print(o1.size())
print(o2.size())

torch.Size([10, 32, 222, 222])
torch.Size([10, 64, 110, 110])


## optimizer

In [8]:
class resnetx(nn.Module):
    
    def __init__(self, num_classes):
        
        super(resnetx, self).__init__()
        
        model = models.resnet18()
        
        self.base = nn.Sequential(*list(model.children())[:-1])
        
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        
        n = x.size()[0]
        
        out = self.base(x)
        out = self.fc( out.view(n, -1) )
        
        return out

In [9]:
net = resnetx(num_classes=10)

In [10]:
crit = nn.CrossEntropyLoss()
opt = optim.SGD([{'params': net.base.parameters(), 'lr': 0.002}, {'params': net.fc.parameters()}], lr=0.02)
scheduler = optim.lr_scheduler.StepLR(optimizer=opt, step_size=5, gamma=0.1)

In [11]:
net.train()

for e in range(1):
    
    for i, (img, lab) in enumerate(dataloader):
        
        img = Variable(img.float())
        lab = Variable(lab.long())
        
        logits = F.softmax( net(img), dim=1 )
        
        loss = crit(logits, lab)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        print(loss.cpu().data.numpy()[0])
    
    scheduler.step()
    
# torch.save( net.state_dict(), '')

2.3010736
2.2906392
2.301759
2.3183694
2.3136604


## gpu

In [18]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

False
0


In [ ]:
nn.parallel.data_parallel(net, data, device_ids=[])

## torchvision

In [23]:
import torchvision.models as models
import torchvision.utils as vutils
import torchvision.transforms.functional as vF
import torchvision.transforms as vtransforms

In [26]:
mm = models.resnet101()

In [ ]:
vutils.save_image()

## decorator

In [35]:
def logger(func):
    def wrapper(*args, **kwargs):
        print('--')
        return func(*args)
    
    return wrapper

In [36]:
@logger
def funcx(x):
    print(x*2)

In [37]:
funcx(10)

--
20


In [116]:
class Solver(object):
    
    def __init__(self, ):
        
        self.event_handler = {}
        
        self.x = 10
        self.y = 20
    
    def add_event_handler(self, name, f, *args, **kwargs):
        
        self.event_handler[name] = ( f, args, kwargs )
        
        
    def on(self, name, *args, **kwargs):
        
        def wrapper(f):
            
            self.add_event_handler(name, f, *args, **kwargs)
            
            return wrapper
        
        return wrapper
    
    
    def run(self):
        
        f, args, kwargs = self.event_handler['x']
            
        f(self, *args, **kwargs)

In [117]:
solver = Solver()

In [121]:
@solver.on('x')

def funcx(solver):
    
    z = solver.x + solver.y
    
    print(z)
    
    return z

In [122]:
solver.event_handler

{'x': (<function __main__.funcx>, (), {})}

In [123]:
solver.run()

30
